In [68]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from efficientnet.tfkeras import EfficientNetB0

In [69]:
data_dir = 'dataset'
output_dir = 'TestRest'

Map class names to labels


In [70]:
class_labels = {
    'WartsMolluscum': 1,
    'UrticariaHives': 2,
    'NormalSkin': 3
}

Create lists to store image paths and corresponding labels

In [71]:
file_paths = []
labels = []

Iterate through the subfolders and collect image paths and labels

In [73]:
for class_name, label in class_labels.items():
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        file_paths.append(os.path.join(class_dir, image_name))
        labels.append(label)

Split the data

In [74]:
train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)
train_paths, valid_paths, train_labels, valid_labels = train_test_split(train_paths, train_labels, test_size=0.2, random_state=42)

Create DataFrames and save them as CSV

In [75]:
train_df = pd.DataFrame({'Image_Path': train_paths, 'Label': train_labels})
test_df = pd.DataFrame({'Image_Path': test_paths, 'Label': test_labels})
valid_df = pd.DataFrame({'Image_Path': valid_paths, 'Label': valid_labels})

Define the paths for output CSV files and the output directory

In [76]:
output_dir = 'E:\\SLIIT\\Y4S1\\TestRest'
train_csv_path = os.path.join(output_dir, 'train.csv')
test_csv_path = os.path.join(output_dir, 'test.csv')
valid_csv_path = os.path.join(output_dir, 'valid.csv')

# Creating the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# save the CSV files
train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)
valid_df.to_csv(valid_csv_path, index=False)


Define image data generators with augmentation for training and validation

In [77]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

 convert the values in the 'Label' column into strings

In [ ]:
train_df['Label'] = train_df['Label'].astype(str)

In [80]:
batch_size = 32
train_generator = datagen.flow_from_dataframe(
    train_df,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),  # Adjust this size according to your model's input size
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1368 validated image filenames belonging to 3 classes.


In [81]:
test_df['Label'] = test_df['Label'].astype(str)

In [82]:
test_generator = datagen.flow_from_dataframe(
    test_df,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 428 validated image filenames belonging to 3 classes.


In [84]:
valid_df['Label'] = valid_df['Label'].astype(str)

In [85]:
valid_generator = datagen.flow_from_dataframe(
    valid_df,
    x_col='Image_Path',
    y_col='Label',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 342 validated image filenames belonging to 3 classes.


creating a base model using EfficientNetB0 architecture

In [91]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

 building additional layers

In [92]:
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(3, activation='softmax')(x)

In [93]:
model = keras.models.Model(inputs=base_model.input, outputs=predictions)

In [94]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Train the model using the training and validation data generators.

In [95]:
epochs = 10
model.fit(train_generator, validation_data=valid_generator, epochs=epochs)

Epoch 1/10
43/43 [==============================] - 226s 5s/step - loss: 0.2826 - accuracy: 0.8801 - val_loss: 2.2001 - val_accuracy: 0.6667
Epoch 2/10
43/43 [==============================] - 196s 5s/step - loss: 0.1259 - accuracy: 0.9554 - val_loss: 0.4189 - val_accuracy: 0.8655
Epoch 3/10
43/43 [==============================] - 193s 4s/step - loss: 0.0442 - accuracy: 0.9876 - val_loss: 0.8205 - val_accuracy: 0.7749
Epoch 4/10
43/43 [==============================] - 173s 4s/step - loss: 0.0443 - accuracy: 0.9846 - val_loss: 0.3582 - val_accuracy: 0.8743
Epoch 5/10
43/43 [==============================] - 175s 4s/step - loss: 0.0839 - accuracy: 0.9751 - val_loss: 0.1689 - val_accuracy: 0.9415
Epoch 6/10
43/43 [==============================] - 175s 4s/step - loss: 0.0608 - accuracy: 0.9795 - val_loss: 0.6106 - val_accuracy: 0.8538
Epoch 7/10
43/43 [==============================] - 173s 4s/step - loss: 0.0642 - accuracy: 0.9795 - val_loss: 0.5383 - val_accuracy: 0.8626
Epoch 8/10
43

Evaluate the model

In [96]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


14/14 [==============================] - 17s 1s/step - loss: 0.1711 - accuracy: 0.9509
Test Accuracy: 95.09%
